In [253]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import re
plt.style.use('ggplot')

In [254]:
df = pd.read_csv('../data.csv')

In [255]:
df.head()

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
0,0,alfa-romeo,Alfa Romeo GTV,red,10/1995,1995,1300,148,201,Manual,Petrol,"10,9 l/100 km",260 g/km,160500.0,2.0 V6 TB
1,1,alfa-romeo,Alfa Romeo 164,black,02/1995,1995,24900,191,260,Manual,Petrol,NaN,- (g/km),190000.0,"Q4 Allrad, 3.2L GTA"
2,2,alfa-romeo,Alfa Romeo Spider,black,02/1995,1995,5900,110,150,Unknown,Petrol,NaN,- (g/km),129000.0,ALFA ROME 916
3,3,alfa-romeo,Alfa Romeo Spider,black,07/1995,1995,4900,110,150,Manual,Petrol,"9,5 l/100 km",225 g/km,189500.0,2.0 16V Twin Spark L
4,4,alfa-romeo,Alfa Romeo 164,red,11/1996,1996,17950,132,179,Manual,Petrol,"7,2 l/100 km",- (g/km),96127.0,"3.0i Super V6, absoluter Topzustand !"


In [256]:
df[df.duplicated( ['brand', 'model', 'color', 'registration_date', 'year',
       'price_in_euro', 'power_kw', 'power_ps', 'transmission_type',
       'fuel_type', 'fuel_consumption_l_100km', 'fuel_consumption_g_km',
       'mileage_in_km', 'offer_description'])].tail(3)

,Unnamed: 0,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
251039,251039,volvo,Volvo V60,black,05/2023,2023,53188,145,197,Automatic,Diesel,"4,7 l/100 km",151 g/km,10.0,B4 D Ultimate Dark Aut. Pano HUD WinterP 20Z
251040,251040,volvo,Volvo V60,black,05/2023,2023,53188,145,197,Automatic,Diesel,"4,7 l/100 km",151 g/km,10.0,B4 D Ultimate Dark Aut. Pano HUD WinterP 20Z
251053,251053,volvo,Volvo V60,black,05/2023,2023,52988,145,197,Automatic,Diesel,"4,7 l/100 km",151 g/km,10.0,B4 D Ultimate Dark Aut. Pano HUD WinterP 20Z


In [257]:
# entferne Duplikate  
df = df.drop_duplicates(subset= ['brand', 'model', 'color', 'registration_date', 'year',
'price_in_euro', 'power_kw', 'power_ps', 'transmission_type',
'fuel_type', 'fuel_consumption_l_100km', 'fuel_consumption_g_km',
'mileage_in_km', 'offer_description']).reset_index(drop= True).copy()

# Droppe zweite Index Spalte
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

In [258]:
 # Zeilen mit falschen Jahreszahlen werden herausgenommen
yearsToFilter = list(df['year'].unique()[:29])
filt = [val in yearsToFilter for val in df['year']]
df = df[filt]

jetzt cleaner DF ohne verschobene Werte

In [259]:
df = df.loc[df['fuel_type'].isin(['Diesel', 'Petrol'])].copy()

In [260]:
df['fuel_consumption_g_km'].value_counts()

fuel_consumption_g_km
- (g/km)             33936
0 g/km                5125
119 g/km              4508
114 g/km              3722
139 g/km              3253
                     ...  
24 g/km                  1
318 km Reichweite        1
156,5 g/km               1
14 g/km                  1
52 g/km                  1
Name: count, Length: 1006, dtype: int64

In [261]:
df[df['fuel_consumption_g_km'] == '318 km Reichweite']

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
151668,opel,Opel Mokka,green,05/2021,2021,27980,100,136,Automatic,Petrol,NaN,318 km Reichweite,19500.0,e Ultimate +NAVI+LED+RFK+PDC+SITZH+LENKH+


In [262]:
df = df.loc[df['fuel_consumption_g_km'].str.contains(r'g/km', na=False)]

In [263]:
df[df['fuel_consumption_g_km'] == '318 km Reichweite']

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description


In [264]:
def fix_model_brand_conflicts(df):
    '''Diese Funktion überprüft, ob es Zeilen gibt, in denen brand = model ist. In diesen Zeilen haben wir keine Informationen
    über das Model. 
    
    Um die Zeilen aber nicht direkt zu droppen, wird vorher geschaut, ob man über bestimmte Spalten das Model eindeutig zuornden kann.
    Ist eine eindeutige Zuordnung möglich, dann überschreiben wir die ursprüngliche Ausprägung in model. Ist keine Zuordnung möglich, dann wird die Zeile
    gedroppt.
    '''

    def normalize(text):
        if pd.isna(text):
            return ""
        return re.sub(r'[^a-z0-9]', '', text.lower())

    df.loc[:, 'brand_norm'] = df['brand'].apply(normalize)
    df.loc[:, 'model_norm'] = df['model'].apply(normalize)

    mask_same = df['brand_norm'] == df['model_norm']

    problem_rows = df[mask_same].copy()
    clean_rows = df[~mask_same].copy()

    grouped_models = clean_rows \
        .groupby(['brand', 'power_ps', 'fuel_consumption_g_km', 'transmission_type', 'fuel_type'])['model'] \
        .unique().reset_index() 

    grouped_models = grouped_models[grouped_models['model'].apply(len) == 1] # nur kontexte bei denen model unique ist (ein element in der liste)
    grouped_models['model'] = grouped_models['model'].apply(lambda x: x[0]) # nimm nur das erste element aus der liste

    problem_fixed = problem_rows.merge(grouped_models, on=['brand', 'power_ps', 'fuel_consumption_g_km', 'transmission_type', 'fuel_type'],
                                       how='left', suffixes=('', '_fixed'))

    recovered = problem_fixed[problem_fixed['model_fixed'].notna()].copy()
    recovered['model'] = recovered['model_fixed']
    recovered = recovered.drop(columns=['model_fixed'])

    final_df = pd.concat([clean_rows, recovered], ignore_index=True) \
                 .drop(columns=['brand_norm', 'model_norm'])

    return final_df

In [265]:
df = fix_model_brand_conflicts(df)

In [266]:
df.isnull().sum()

brand                           0
model                           0
color                          71
registration_date               0
year                            0
price_in_euro                   0
power_kw                       72
power_ps                       72
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    17403
fuel_consumption_g_km           0
mileage_in_km                  29
offer_description               0
dtype: int64

In [267]:
df.query('model == "Alfa Romeo Giulietta" & year == "2012" & power_ps == "120"')

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
418,alfa-romeo,Alfa Romeo Giulietta,red,03/2012,2012,7490,88,120,Manual,Petrol,"7,2 l/100 km",164 g/km,121650.0,1.4 TB 16V KLIMAAUTOMATIK/PDC/8xALU
421,alfa-romeo,Alfa Romeo Giulietta,red,06/2012,2012,7600,88,120,Manual,Petrol,NaN,0 g/km,123612.0,1.4 TB 16V
426,alfa-romeo,Alfa Romeo Giulietta,black,06/2012,2012,5500,88,120,Manual,Petrol,"6,6 l/100 km",156 g/km,182457.0,1.4
428,alfa-romeo,Alfa Romeo Giulietta,black,05/2012,2012,4790,88,120,Manual,Petrol,"6,4 l/100 km",152 g/km,176835.0,Super KLIMA ALU
436,alfa-romeo,Alfa Romeo Giulietta,black,03/2012,2012,6490,88,120,Manual,Petrol,"6,4 l/100 km",152 g/km,152000.0,Super (191)
438,alfa-romeo,Alfa Romeo Giulietta,black,03/2012,2012,7200,88,120,Manual,Petrol,"6,4 l/100 km",152 g/km,142321.0,Super
442,alfa-romeo,Alfa Romeo Giulietta,black,04/2012,2012,7000,88,120,Manual,Petrol,"6,4 l/100 km",149 g/km,170000.0,1.4 TB 16V
443,alfa-romeo,Alfa Romeo Giulietta,black,09/2012,2012,6400,88,120,Manual,Petrol,"6,6 l/100 km",156 g/km,113873.0,Turismo (191)
450,alfa-romeo,Alfa Romeo Giulietta,white,03/2012,2012,8485,88,120,Manual,Petrol,"6,4 l/100 km",- (g/km),76370.0,Giulietta 1.4 TB 16V Super


In [268]:
def clean_fuel_consumption(value): # Bei Elektroautos steht Reichweite
    if pd.isna(value) or 'l/100 km' not in str(value):
        return np.nan
    try:
        return float(value.split(' ')[0].replace(',', '.'))
    except:
        return np.nan
                    
def clean_fuel_consumption_g(value):
    if pd.isna(value) or value == '- (g/km)' or 'g/km' not in str(value): # Bei Elektroautos steht Reichweite
        return np.nan
    try:
        return float(value.split(' ')[0])
    except:
        return np.nan

In [269]:
df['fuel_consumption_l_100km'] = df['fuel_consumption_l_100km'].apply(clean_fuel_consumption)
df['fuel_consumption_g_km'] = df['fuel_consumption_g_km'].apply(clean_fuel_consumption_g)    

In [270]:
def calculate_fuel_consumption(row):

        conversion_factor = 0.043103448275862

        if pd.isna(row['fuel_consumption_l_100km']) or row['fuel_consumption_l_100km'] == 0:
            if pd.notna(row['fuel_consumption_g_km']) and row['fuel_consumption_g_km'] != 0:
                return row['fuel_consumption_g_km'] * conversion_factor
            else:
                return np.nan
        else:
            return row['fuel_consumption_l_100km']

df['fuel_consumption_l_100km'] = df.apply(calculate_fuel_consumption, axis=1)

In [271]:
df.isnull().sum()

brand                           0
model                           0
color                          71
registration_date               0
year                            0
price_in_euro                   0
power_kw                       72
power_ps                       72
transmission_type               0
fuel_type                       0
fuel_consumption_l_100km    16224
fuel_consumption_g_km       34935
mileage_in_km                  29
offer_description               0
dtype: int64

In [272]:
def impute_fuel_staged(df, target_col):
    groupings = [
        ['brand', 'model', 'year', 'power_ps', 'fuel_type', 'transmission_type'],  # Stufe 1: eindeutig
        ['brand', 'model', 'power_ps', 'fuel_type', 'transmission_type'],          # Stufe 2: Mittelwert
        ['model', 'fuel_type']                                                     # Stufe 3: fallback
    ]
    
    # Stufe 1: nur eindeutige Fälle (nunique == 1)
    group_cols = groupings[0]
    counts = df.groupby(group_cols)[target_col].nunique().reset_index(name='unique_count')
    unique_contexts = counts.query('unique_count == 1').drop(columns='unique_count')

    fuel_map_unique = (
        df.dropna(subset=[target_col])
          .merge(unique_contexts, on=group_cols, how='inner')
          .groupby(group_cols)[target_col]
          .first()
    )

    def fill_unique(row):
        if pd.isna(row[target_col]):
            key = tuple(row[col] for col in group_cols)
            return fuel_map_unique.get(key, np.nan)
        return row[target_col]

    df[target_col] = df.apply(fill_unique, axis=1)

    # Stufe 2 & 3: falls noch NaNs, mit mean
    for group_cols in groupings[1:]:
        fuel_map_mean = (
            df.dropna(subset=[target_col])
              .groupby(group_cols)[target_col]
              .mean()
              .round(2)
        )

        def fill_mean(row):
            if pd.isna(row[target_col]):
                key = tuple(row[col] for col in group_cols)
                return fuel_map_mean.get(key, np.nan)
            return row[target_col]

        df[target_col] = df.apply(fill_mean, axis=1)

    return df

In [273]:
df = impute_fuel_staged(df, target_col='fuel_consumption_l_100km')
df = impute_fuel_staged(df, target_col='fuel_consumption_g_km')

In [274]:
df.isnull().sum()

brand                         0
model                         0
color                        71
registration_date             0
year                          0
price_in_euro                 0
power_kw                     72
power_ps                     72
transmission_type             0
fuel_type                     0
fuel_consumption_l_100km     98
fuel_consumption_g_km       144
mileage_in_km                29
offer_description             0
dtype: int64

In [275]:
df[df['model'] == 'Renault Master']

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
151423,renault,Renault Master,blue,03/2001,2001,3800,59,80,Manual,Diesel,10.30,201.69,145000.0,2.5 D LKW zu
151649,renault,Renault Master,grey,12/2005,2005,7500,73,99,Manual,Diesel,8.86,201.69,89000.0,"L1H1 3,5t Ka II Phase 2 Kasten"
151737,renault,Renault Master,grey,06/2006,2006,6500,73,99,Manual,Diesel,8.86,201.69,123000.0,"L1H1 3,5t Ka II Phase 2 Kasten"
151860,renault,Renault Master,grey,05/2007,2007,10950,88,120,Semi-automatic,Diesel,9.10,241.00,296000.0,"II Phase 2 L1 2,8t Wohnmobil"
151940,renault,Renault Master,blue,04/2008,2008,4500,88,120,Manual,Diesel,8.80,232.00,204500.0,Master 2.5 dCi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158555,renault,Renault Master,white,02/2023,2023,33540,99,135,Manual,Diesel,9.50,250.00,20.0,"L3H2 3,5t Kasten 135 dCi *Klima*GRA*PDC*"
158556,renault,Renault Master,white,02/2023,2023,35980,100,136,Manual,Diesel,9.10,238.00,1001.0,"Kasten L2H2 135 3,3t Klima Bluetooth AHK"
158557,renault,Renault Master,white,04/2023,2023,37150,100,136,Manual,Diesel,7.90,198.00,10.0,"L3H2 HKa 3,5t+SOFORT+KAMERA+NAVI+TEMP+DAB"
158558,renault,Renault Master,white,03/2023,2023,39674,110,150,Manual,Diesel,9.20,242.00,10.0,"FWD Kasten KOMFORT L2H2 3,5t ENERGY Blue"


In [276]:
df[df['fuel_consumption_l_100km'].isna() | df['fuel_consumption_g_km'].isna()]

,brand,model,color,registration_date,year,price_in_euro,power_kw,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,fuel_consumption_g_km,mileage_in_km,offer_description
14,alfa-romeo,Alfa Romeo 155,grey,06/1997,1997,7950,88,120,Manual,Petrol,8.4,NaN,101726.0,"1.6l Twin Spark, Erstlack,sehr gepflegt"
1419,aston-martin,Aston Martin Lagonda,green,01/2018,2018,859990,412,560,Automatic,Petrol,NaN,NaN,5150.0,Taraf
1565,audi,Audi 80,silver,11/1998,1998,8999,92,125,Manual,Petrol,10.8,NaN,104600.0,Cabrio 1.8 Lederausst/Klimaanlage/Sitzheizung
1579,audi,Audi 80,black,06/1999,1999,9900,92,125,Manual,Petrol,10.8,NaN,172000.0,Cabrio LEDER KLIMA SITZHEIZUNG
1667,audi,Audi 80,blue,07/2000,2000,9500,128,174,Manual,Petrol,10.8,NaN,249641.0,"Audi 80 Cabriolet 2.8 - V6, Klima, TÜV NEU"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209649,volkswagen,Volkswagen Atlas,grey,02/2018,2018,39900,206,280,Automatic,Petrol,NaN,NaN,98175.0,"3,6 V-6 4x4 Panoramadach"
212037,volkswagen,Volkswagen Atlas,grey,02/2019,2019,36500,208,283,Automatic,Petrol,NaN,NaN,87700.0,3.6L V6 4 Motion SE R-LINE
219129,volvo,Volvo 850,blue,09/1995,1995,700,106,144,Manual,Petrol,NaN,NaN,367462.0,850 2.5-10V
219130,volvo,Volvo 850,blue,06/1995,1995,3590,106,144,Automatic,Petrol,NaN,NaN,280000.0,2.5-10V


In [277]:
df.drop(columns=['fuel_consumption_g_km'], axis = 1)
    
# Spalten ins numerische umwandeln
for col in ['power_ps', 'power_kw', 'mileage_in_km', 'price_in_euro']:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    
# Encoding vom Datum ins numerische
df['registration_date'] = pd.to_datetime(df['registration_date'], format='%m/%Y', errors='coerce')
df['registration_month'] = df['registration_date'].dt.month
df['registration_year'] = df['registration_date'].dt.year

df = df.drop(['registration_date', 'year','power_kw', 'offer_description', 'fuel_consumption_g_km'], axis=1) # year sonst zweimal drinne

In [278]:
df = df.dropna().reset_index(drop= True)

In [279]:
df

,brand,model,color,price_in_euro,power_ps,transmission_type,fuel_type,fuel_consumption_l_100km,mileage_in_km,registration_month,registration_year
0,alfa-romeo,Alfa Romeo GTV,red,1300,201.0,Manual,Petrol,10.90,160500.0,10,1995
1,alfa-romeo,Alfa Romeo 164,black,24900,260.0,Manual,Petrol,10.30,190000.0,2,1995
2,alfa-romeo,Alfa Romeo Spider,black,5900,150.0,Unknown,Petrol,9.54,129000.0,2,1995
3,alfa-romeo,Alfa Romeo Spider,black,4900,150.0,Manual,Petrol,9.50,189500.0,7,1995
4,alfa-romeo,Alfa Romeo 164,red,17950,179.0,Manual,Petrol,7.20,96127.0,11,1996
...,...,...,...,...,...,...,...,...,...,...,...
222648,volkswagen,Volkswagen Polo,red,12900,95.0,Manual,Diesel,4.00,134000.0,1,2019
222649,volkswagen,Volkswagen T6.1 Multivan,silver,30980,150.0,Manual,Diesel,7.40,101000.0,6,2021
222650,volkswagen,Volkswagen T7 Multivan,red,54990,204.0,Automatic,Petrol,8.90,21080.0,4,2022
222651,volkswagen,Volkswagen T7 Multivan,red,72990,150.0,Automatic,Petrol,0.80,9000.0,1,2023


In [280]:
def preprocess_new():

    # load the data 
    df = pd.read_csv('../data.csv')

    # entferne Duplikate  
    df = df.drop_duplicates(subset= ['brand', 'model', 'color', 'registration_date', 'year',
    'price_in_euro', 'power_kw', 'power_ps', 'transmission_type',
    'fuel_type', 'fuel_consumption_l_100km', 'fuel_consumption_g_km',
    'mileage_in_km', 'offer_description']).reset_index(drop= True).copy()

    # Droppe zweite Index Spalte
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)

     # Zeilen mit falschen Jahreszahlen werden herausgenommen
    yearsToFilter = list(df['year'].unique()[:29])
    filt = [val in yearsToFilter for val in df['year']]
    df = df[filt]

    df = df.loc[df['fuel_type'].isin(['Diesel', 'Petrol'])].copy()
    # Verbleibende Fahrzeuge die zwar Petrol und Diesel sind aber nicht g/km haben sondern 
    # Reichweite etc. 
    df = df.loc[df['fuel_consumption_g_km'].str.contains(r'g/km', na=False)]

    # Try to fix rows where brand = model
    df = fix_model_brand_conflicts(df)
    # convert to numeric
    df['fuel_consumption_l_100km'] = df['fuel_consumption_l_100km'].apply(clean_fuel_consumption)
    df['fuel_consumption_g_km'] = df['fuel_consumption_g_km'].apply(clean_fuel_consumption_g) 

    # calculate fuel_consumption using formula
    df['fuel_consumption_l_100km'] = df.apply(calculate_fuel_consumption, axis=1)


    df = impute_fuel_staged(df, target_col='fuel_consumption_l_100km')
    df = impute_fuel_staged(df, target_col='fuel_consumption_g_km')

    df.drop(columns=['fuel_consumption_g_km'], axis = 1)
        
    # Spalten ins numerische umwandeln
    for col in ['power_ps', 'power_kw', 'mileage_in_km', 'price_in_euro']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        
    # Encoding vom Datum ins numerische
    df['registration_date'] = pd.to_datetime(df['registration_date'], format='%m/%Y', errors='coerce')
    df['registration_month'] = df['registration_date'].dt.month
    df['registration_year'] = df['registration_date'].dt.year

    df = df.drop(['registration_date', 'year','power_kw', 'offer_description', 'fuel_consumption_g_km'], axis=1) # year sonst zweimal drinne


    df = df.dropna().reset_index(drop= True)

    return df